In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

# Spark 

In [2]:
SPARK_CONFIG["process_label"] = "story_popular_model_check_products"
SPARK_CONFIG["numofcores"] = 8

In [3]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

# load table from ISKRA 

In [5]:
db = OracleDB('iskra4')
ISKRA_TABLE = ISKRA_TABLE_NAME

In [6]:
ISKRA_TABLE

'ISKRA.recsys_story_asup_ratings'

In [7]:
story_ratings_iskra = hive.read.format('jdbc')\
                          .option('url', 'jdbc:oracle:thin:@//' + db.dsn) \
                          .option('user', db.user) \
                          .option('password', db.password) \
                          .option('dbtable', ISKRA_TABLE) \
                          .option('driver', 'oracle.jdbc.driver.OracleDriver')\
                     .load().cache()

In [6]:
story_ratings_sbx = load_table(SBX_TEAM_DIGITCAMP, ALS_ASUP_RATINGS_SCALLED, hive).cache()

In [10]:
story_ratings_sbx = story_ratings_sbx.withColumn("prod_cd_asup", f.upper(f.col("prod_cd_asup")))

In [ ]:
story_ratings_sbx.show()

In [12]:
%%time
story_ratings_sbx.count()

CPU times: user 1.27 ms, sys: 305 µs, total: 1.57 ms
Wall time: 1.2 s


329214260

In [13]:
story_ratings_sbx.select("load_dt").distinct().show()

,load_dt
0,2022-02-10


In [14]:
all_users_cnt = story_ratings_sbx.select("inn").distinct().count()

In [15]:
all_users_cnt

5431034

# product_list 

In [16]:
prod_list = ["leasing",
             "guarantee",
             "rabota_ru",
             "merchant-acquiring",
             "zarplata",
             "evotor",
             "e-acquiring",
             "mybuch",
             "credit_card",
             "bip",
             "lyurist",
             "kredit",
             "sberrating"]

prod_list_up = [s.upper() for s in prod_list]

In [5]:
prod_list_up

['LEASING',
 'GUARANTEE',
 'RABOTA_RU',
 'MERCHANT-ACQUIRING',
 'ZARPLATA',
 'EVOTOR',
 'E-ACQUIRING',
 'MYBUCH',
 'CREDIT_CARD',
 'BIP',
 'LYURIST',
 'KREDIT',
 'SBERRATING']

In [17]:
n = 1
for prod in prod_list_up:
    prod_inn_cnt = story_ratings_sbx.filter("prod_cd_asup like '{prod_name}'".format(prod_name=prod))\
                                    .select("inn").distinct().count()
    print(n, prod, ":", prod_inn_cnt)
    n+=1

print("\n\ninn_cnt :", all_users_cnt)

1 LEASING : 5431034
2 GUARANTEE : 5431034
3 RABOTA_RU : 5431034
4 MERCHANT-ACQUIRING : 5431034
5 ZARPLATA : 5431034
6 EVOTOR : 5431034
7 E-ACQUIRING : 5431034
8 MYBUCH : 5431034
9 CREDIT_CARD : 5431034
10 BIP : 5431034
11 LYURIST : 5431034
12 KREDIT : 5431034
13 SBERRATING : 5431034


inn_cnt : 5431034


In [12]:
pop_model = load_table(SBX_TEAM_DIGITCAMP, "RECSYS_STORY_POPULAR_MODEL", hive).cache()

In [15]:
pop_model.filter("prod_cd_asup like '%ris%'").show()

,prod_cd_asup,ratings,load_dt
0,lyurist,0.001032,2022-02-10


In [36]:
story_ratings_sbx.filter((f.col("ratings") >= 5) | (f.col("prod_cd_asup").isin(prod_list))).select("prod_cd_asup").distinct().show(100)

,prod_cd_asup
0,merchant-acquiring
1,sberrating
2,kredit
3,mybuch
4,guarantee
5,e-acquiring
6,evotor
7,LYurist
8,bip
9,credit_card
